# Serer's Triplets Theorem

### This code implements an approach described in the article: "Serer's Triplets Theorem: A Polynomial Method for Mapping Arbitrary Graphs"

The central idea is:
- Represent all valid 3-vertex subpaths (triplets) of a graph.
- Concatenate these triplets using sliding windows over common vertices.
- Reconstruct all possible paths that traverse the vertices while preserving connectivity.

In [ ]:
pip install importnb # to import notebooks as modules --- IGNORE ---

# Import some graphs for tests implementations

### See the graphs.ipynb file

In [2]:
from importnb import Notebook

with Notebook():
    import graphs  # importa "graphs.ipynb"

# Auxiliar methods and declarations

In [9]:
from itertools import permutations, combinations
import itertools

# === put your selected graph here' ===
graph_selected = graphs.digraph1 # digraph1 is selected
len_graph_vertices = len(graph_selected)

# --- Auxiliar Maps ---
index_to_label = {i: label for label, i, _ in graph_selected}
label_to_index = {label: i for label, i, _ in graph_selected}
adj = {i: viz for _, i, viz in graph_selected}  # adjacency list
neighbors = adj  # only renaming for clarity


# --- Function: verify if a permutation is a valid path ---
def is_valid_path(perm):
    """Return True if all consecutive vertices in perm are adjacent."""
    for a, b in zip(perm, perm[1:]):
        if b not in adj[a]:
            return False
    return True


# Construction of the Family of Valid Triplets J

This function generates all permutations for each current combination. In this way, it ensures that all possible combinations are sent to the *is_valid_path()* function,  
so that no potential triplet is disregarded.  

This process fully satisfies the formal definition of J:

$$ 
    J = \{(v_i, v_j, v_k) \mid (v_i, v_j) \in E, \ (v_j, v_k) \in E, \; i \neq j, \; j \neq k, \; k \neq i \}.
$$

where J contains all valid triplets of the graph.


In [4]:
def construct_j_set(len_graph_vertices):
    valid_triplets = []

    for comb in combinations(range(1, len_graph_vertices + 1), 3):
        # print("Actual Combination: ", (comb))                                 # Uncomment to see actual combination
        for perm in permutations(comb):
            # print(perm)                                                       # Uncomment to see all permutations fo actual combination
            if is_valid_path(perm):
                valid_triplets.append(list(perm))
                # Add the reversed triplet as well
                reversed_triplet = list(reversed(perm))
                if is_valid_path(reversed_triplet):
                    valid_triplets.append(reversed_triplet)                     # now like indices

    return valid_triplets

# Construction of Triplets Adjacency ADJ

Generates the Cartesian product of $J \times J $, resulting in a **directed graph** of valid paths. In the final algorithm, it performs a verification of the adjacency condition, where:  

- the last two vertices of $t_1$ must be equal to the first two vertices of $t_2$, and  
- the last vertex of $t_2$ must be a neighbor of the last vertex of $t_1$.  

In this way, it satisfies the formal definition:

$$
    ADJ = \{(A, B) \in J \times J \mid 
    A = (v_i, v_j, v_k), \ B = (v_j, v_k, v_l), \ 
    v_l \notin \{v_i, v_j, v_k\}, \ (v_k, v_l) \in E \}.
$$


In [5]:
def build_adj(triples):
    adj_triples = {i: [] for i in range(len(triples))}

    # product generates all (i, j) index pairs
    for i, j in itertools.product(range(len(triples)), repeat=2):
        if i == j:
            continue

        t1, t2 = triples[i], triples[j]

        # Adjacency condition:
        if t1[1] == t2[0] and t1[2] == t2[1] and t2[2] not in t1:
            if t2[2] in neighbors[t1[2]]:
                adj_triples[i].append(j)

    return adj_triples


# Construction of Valid Paths

This function iterates over the $ADJ$ set, marking the current triplet as *"current"*, and verifying on the digraph all its connections in a greedy manner. Since this method uses every   
triplet as a starting point, no main valid path is left out of the final function output, fully satisfying the definition:

$$
    (A_j, A_{j+1}) \in ADJ, \quad \forall\, 1 \le j < k,
$$

where:

$$
    A_1 \oplus A_2 \oplus \dots \oplus A_k = (v_1, v_2, \dots, v_{k+2}).
$$


In [6]:
def build_paths(triples):
    adj_triples = build_adj(triples)
    paths = []
    seen_paths = set()  # to avoid duplicate paths

    # print("Digraph of adjacency of Triplets (ADJ): ", adj_triples)             # Uncomment to see adjacency of triplets

    for start_idx in range(len(triples)):
        path = triples[start_idx].copy()
        current_idx = start_idx

        while True:
            extended = False
            for next_idx in adj_triples[current_idx]:
                next_triple = triples[next_idx]
                next_vertex = next_triple[2]
                if next_vertex not in path:  # no revisiting vertices
                    path.append(next_vertex)
                    current_idx = next_idx
                    extended = True
                    break
            if not extended:
                break

        # Convert tuple to set to avoid duplicates
        path_tuple = tuple(path)
        if path_tuple not in seen_paths:
            seen_paths.add(path_tuple)
            paths.append(path)

    return paths

# Running the Code

In [7]:
# Generation of all valid triplets
J = construct_j_set(len_graph_vertices)

# --- Construction of some valid Paths ---
paths = build_paths(J)

# Show paths with vertex names
for p in paths:
    print([index_to_label[i] for i in p])

['A', 'B', 'C', 'D', 'E', 'F']
['B', 'C', 'D', 'E', 'F']
['B', 'C', 'x3']
['C', 'D', 'E', 'F']
['C', 'D', 'x4']
['D', 'E', 'F']
['D', 'E', 'x5']
